In [1]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time
import logging
import pandas as pd
import os

import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils
import src.utils.ahn_utils as ahn_utils
import src.utils.bgt_utils as bgt_utils
import src.utils.plot_utils as plot_utils

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

import laspy
import open3d as o3d
from src.utils.interpolation import FastGridInterpolator
from searchspace_reducer import SearchSpaceReducer

import glob
from src.analysis import analysis_tools
from sklearn.metrics import classification_report

logger = logging.getLogger(__name__)

In [9]:
# Load data.
tilecode = '2387_9699'

# Labelled LAS file (see Notebook 0).
las_file = '../datasets/Lairessestraat/pointcloud/labelled_' + tilecode + '.laz'
reduced_las_file = '../datasets/Lairessestraat/processed/M2_' + tilecode + '.laz'

# AHNReader for elevation data.
ahn_data_folder = '../datasets/Lairessestraat/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)
ahn_tile = ahn_reader.filter_tile(tilecode)

# BGTReader
bgt_building_file = '../datasets/Lairessestraat/bgt/bgt_buildings.csv'
bgt_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_building_file)

DEBUG - Caching enabled.


In [4]:
# Load pointcloud
pointcloud = las_utils.read_las(las_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

# colors
pc_colors =  np.vstack((pointcloud.red, pointcloud.blue, pointcloud.green)).T

# Mask
mask = np.ones(len(points), dtype=bool)

In [36]:
SSR = SearchSpaceReducer(0, ahn_tile, bgt_reader)
label_mask, labels = SSR.get_label_mask(points, None, None, tilecode)

In [40]:
# Plot result
plot_utils.plot_pointcloud_o3d(points[label_mask], np.zeros(label_mask.sum()))

#### Batch folder + Stats

In [11]:
def process_file(in_file, ahn_reader, bgt_reader, folder_path, save_output):
        
    tilecode = las_utils.get_tilecode_from_filename(in_file)
    if not os.path.isfile(folder_path + '/ahn/ahn_' + tilecode + '.npz'):
        return

    logger.info(f'Processing file {in_file}.')
    start = time.time()
    pointcloud = las_utils.read_las(in_file)
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    true_labels = np.zeros((len(points),), dtype='uint16')
    true_labels[(10 < pointcloud.label) & (pointcloud.label < 15)] = 11
    true_mask = true_labels == 11

    labels = np.full((len(points),), -1, dtype='uint16')

    ahn_tile = ahn_reader.filter_tile(tilecode)
    SSR = SearchSpaceReducer(0, ahn_tile, bgt_reader, 5, 2)
    label_mask, labels = SSR.get_label_mask(points, None, None, tilecode)
    # labels[label_mask] = SSR.get_label()
    
    labels_pred = np.zeros((len(points),), dtype='uint16')
    labels_pred[label_mask] = 11

    duration = time.time() - start
    stats, report = analysis_tools.get_cable_stats_m1(labels, true_mask)
    report['time'] = duration
    report['tilecode'] = tilecode

    logger.info('STATISTICS\n' + stats)
    logger.info(f'File processed in {duration:.2f}s.')

    if save_output:
        outpath = folder_path + '/processed/M1_' + tilecode + '.laz'
        las_utils.label_and_save_las(pointcloud, labels, outpath)

    return report

    

In [12]:
def process_folder(folder_path, save_output=False):

    # AHNReader for elevation data.
    ahn_data_folder = folder_path + '/ahn/'
    ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

    # BGTReader
    bgt_building_file = folder_path + '/bgt/bgt_buildings.csv'
    bgt_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_building_file)

    results = []

    for file in glob.iglob(folder_path + '/pointcloud/labelled_*.laz'):
        report = process_file(file, ahn_reader, bgt_reader, folder_path, save_output)
        results.append(report)

    return pd.DataFrame(results)



In [13]:
df = process_folder('../datasets/Lairessestraat', True)

DEBUG - Caching enabled.


INFO - 2022-04-26 10:37:58,452 - 4257292774 - Processing file ../datasets/Lairessestraat/pointcloud/labelled_2394_9702.laz.
INFO - 2022-04-26 10:38:00,399 - searchspace_reducer - Search Space Reducer(label=0)
INFO - 2022-04-26 10:38:07,740 - 4257292774 - STATISTICS
Reduced:    4248444 -->     186711 points
Class 0.0, Unlabelled      186711 points ( 4.4 %) /   22140 cable-points (0.988 %)
Class 1.0, Ground         3452426 points (81.3 %) /       0 cable-points (0.000 %)
Class 2.0, Building        609307 points (14.3 %) /     263 cable-points (0.012 %)

INFO - 2022-04-26 10:38:07,741 - 4257292774 - File processed in 2.96s.
INFO - 2022-04-26 10:38:09,418 - 4257292774 - Processing file ../datasets/Lairessestraat/pointcloud/labelled_2388_9700.laz.
INFO - 2022-04-26 10:38:11,747 - searchspace_reducer - Search Space Reducer(label=0)
/Users/FalkeBoskaljon/Documents/Study/Master/Internship/Urban_PointCloud_Processing/milestone_1/searchspace_reducer.py:142: RuntimeWarning: All-NaN slice encounte

In [14]:
df

,start_points,end_points,reduce_per,FN_Ground,FN_Building,f1_score,time,tilecode
0,4248444,186711,0.96,0,263,0.211751,2.962610,2394_9702
1,5769704,491075,0.91,0,106,0.084512,3.375272,2388_9700
2,4853053,508411,0.90,0,390,0.048792,3.160275,2389_9700
3,2194654,162858,0.93,0,36,0.075585,2.080370,2391_9701
4,2788759,151087,0.95,0,43,0.095892,2.233679,2386_9699
5,4379344,92320,0.98,0,0,0.411945,2.969818,2387_9699
6,6508713,509014,0.92,0,0,0.015923,4.261578,2387_9700


#### **Future Idea:**

Interpolate whole tile to remove all points below groound elevation + threshold

In [ ]:
# interpolate.griddata(points, values, (grid_x, grid_y), method='linear')

### **Alternative:** AHN Building Filter (No BGT)

In [6]:
def AHN_building_filter():
    # Building Filter: ONLY AHN
    tilecode = '2407_9765'
    in_file = '../datasets/Kattenslootbrug/pointcloud/proc_ahn4_' + tilecode + '.laz'
    ahn_data_folder = '../datasets/Kattenslootbrug/ahn/'

    # Read Point Cloud
    pointcloud = las_utils.read_las(in_file)
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    labels = np.zeros((len(points),), dtype='uint16')

    # AHN Fuser
    ahn_reader = ahn_utils.NPZReader(ahn_data_folder)
    ahn_tile = ahn_reader.filter_tile(tilecode)
    fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'], ahn_tile['building_surface']) # ..ahn_reader.interpolate()
    target_z = fast_z(points)

    epsilon = 0.2
    label_mask = (points[:, 2] < target_z + epsilon)
    labels[label_mask] = 0 # TARGET_LABEL 

DEBUG - Caching enabled.
